In [1]:
!pip install torch
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from transformers import RobertaTokenizer, TFRobertaModel
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
import xgboost as xgb
from  sklearn.linear_model import RidgeCV

In [5]:
code = pd.read_excel(r"/content/totalmerged.xlsx")
code

,Question,Correct_Code,Code_with_Error,Total_Marks
0,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,7.0
1,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,8.0
2,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,5.0
3,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\n\nvoid printFactors(int nu...,7.0
4,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\n\nvoid printFactors(int nu...,5.0
...,...,...,...,...
995,Print maximum Occurring Character in a String ...,#include <stdio.h>\n\n#define ASCII_SIZE 128\n...,#include <stdio.h> \n\n#define MAX_SIZE 100 \n...,7.0
996,Print maximum Occurring Character in a String ...,#include <stdio.h>\n\n#define ASCII_SIZE 128\n...,#include <stdio.h> \n\nchar getMaxOccurringCha...,6.0
997,Print maximum Occurring Character in a String ...,#include <stdio.h>\n\n#define ASCII_SIZE 128\n...,#include <stdio.h> \n\nchar getMaxOccurringCha...,6.0
998,Print maximum Occurring Character in a String ...,#include <stdio.h>\n\n#define ASCII_SIZE 128\n...,#include <stdio.h> \n\n#define ASCII_SIZE 128 ...,5.0


In [6]:
# Merge columns
code['merged'] = code['Question'] + ' ' + code['Code_with_Error']
code

,Question,Correct_Code,Code_with_Error,Total_Marks,merged
0,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,7.0,Print the factors of a number #include <stdio....
1,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,8.0,Print the factors of a number #include <stdio....
2,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,5.0,Print the factors of a number #include <stdio....
3,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\n\nvoid printFactors(int nu...,7.0,Print the factors of a number #include <stdio....
4,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\n\nvoid printFactors(int nu...,5.0,Print the factors of a number #include <stdio....
...,...,...,...,...,...
995,Print maximum Occurring Character in a String ...,#include <stdio.h>\n\n#define ASCII_SIZE 128\n...,#include <stdio.h> \n\n#define MAX_SIZE 100 \n...,7.0,Print maximum Occurring Character in a String ...
996,Print maximum Occurring Character in a String ...,#include <stdio.h>\n\n#define ASCII_SIZE 128\n...,#include <stdio.h> \n\nchar getMaxOccurringCha...,6.0,Print maximum Occurring Character in a String ...
997,Print maximum Occurring Character in a String ...,#include <stdio.h>\n\n#define ASCII_SIZE 128\n...,#include <stdio.h> \n\nchar getMaxOccurringCha...,6.0,Print maximum Occurring Character in a String ...
998,Print maximum Occurring Character in a String ...,#include <stdio.h>\n\n#define ASCII_SIZE 128\n...,#include <stdio.h> \n\n#define ASCII_SIZE 128 ...,5.0,Print maximum Occurring Character in a String ...


In [7]:
# Fill missing values with empty strings
code['merged'] = code['merged'].fillna('')

In [8]:
# Load the RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
roberta_model = TFRobertaModel.from_pretrained("roberta-base")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

In [9]:
# Tokenize and convert the textual data to vectors
X_text = code["merged"].values
y = code["Total_Marks"].values

In [10]:
X_vectors = []
for text in X_text:
    inputs = tokenizer.encode_plus(text, add_special_tokens=True, return_tensors='tf', max_length=512, truncation=True)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    output = roberta_model(input_ids, attention_mask=attention_mask)[0][:, 0, :]
    X_vectors.append(output.numpy())

In [11]:

X_vectors = np.array(X_vectors)
y = np.array(y)

In [12]:
X_vectors

array([[[-0.04381751,  0.01298407, -0.06823914, ..., -0.01078247,
         -0.06515283,  0.00736107]],

       [[-0.03485733,  0.02759037, -0.06694083, ...,  0.01000899,
         -0.06539921,  0.01887219]],

       [[-0.04260368,  0.00817572, -0.06706099, ..., -0.01272675,
         -0.06602026,  0.00926108]],

       ...,

       [[-0.0207641 ,  0.02080104, -0.06215063, ..., -0.03736268,
         -0.05828968,  0.02422292]],

       [[-0.00893431,  0.01403941, -0.07219768, ..., -0.02958877,
         -0.03848673,  0.0169211 ]],

       [[-0.01983733,  0.01483568, -0.06987625, ..., -0.03227933,
         -0.05074823,  0.02020184]]], dtype=float32)

In [13]:
# Split the data into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X_vectors, y, test_size=0.25, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)


In [14]:
# Build the LSTM model
input_layer = Input(shape=(X_vectors.shape[1], X_vectors.shape[2]))
lstm_layer1 = LSTM(units=128, dropout=0.2, recurrent_dropout=0.2)(input_layer)
output_layer = Dense(units=1, activation='relu')(lstm_layer1)
model = Model(inputs=input_layer, outputs=output_layer)

In [15]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 768)]          0         
                                                                 
 lstm (LSTM)                 (None, 128)               459264    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 459,393
Trainable params: 459,393
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae', 'mape', 'mse'])

In [17]:
# Train the model
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, callbacks=[early_stopping])

Epoch 1/50
18/18 [==============================] - 3s 29ms/step - loss: 24.2113 - mae: 4.3654 - mape: 10812325.0000 - mse: 24.2113 - val_loss: 8.2278 - val_mae: 2.4123 - val_mape: 22539914.0000 - val_mse: 8.2278
Epoch 2/50
18/18 [==============================] - 0s 10ms/step - loss: 5.6774 - mae: 1.9425 - mape: 19009722.0000 - mse: 5.6774 - val_loss: 7.0288 - val_mae: 2.1026 - val_mape: 35638528.0000 - val_mse: 7.0288
Epoch 3/50
18/18 [==============================] - 0s 10ms/step - loss: 5.1781 - mae: 1.8041 - mape: 22684808.0000 - mse: 5.1781 - val_loss: 6.0313 - val_mae: 1.9739 - val_mape: 32338674.0000 - val_mse: 6.0313
Epoch 4/50
18/18 [==============================] - 0s 11ms/step - loss: 4.8011 - mae: 1.7813 - mape: 19893958.0000 - mse: 4.8011 - val_loss: 5.8387 - val_mae: 1.9278 - val_mape: 31067274.0000 - val_mse: 5.8387
Epoch 5/50
18/18 [==============================] - 0s 11ms/step - loss: 4.7930 - mae: 1.7680 - mape: 22274974.0000 - mse: 4.7930 - val_loss: 5.9495 - val

In [18]:
# Evaluate the model on train and test sets
feature_model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
X_train_features = feature_model.predict(X_train)
X_test_features = feature_model.predict(X_test)


8/8 [==============================] - 0s 3ms/step


In [19]:
import lightgbm as lgb
from catboost import CatBoostRegressor

# CatBoost Regressor
catboost_regressor = CatBoostRegressor()
catboost_regressor.fit(X_train_features, y_train)
y_train_pred_catboost = catboost_regressor.predict(X_train_features)
y_test_pred_catboost = catboost_regressor.predict(X_test_features)

# RidgeCv Regressor
ridge_regressor = RidgeCV()
ridge_regressor.fit(X_train_features, y_train)
y_train_pred_ridge = ridge_regressor.predict(X_train_features)
y_test_pred_ridge = ridge_regressor.predict(X_test_features)

# LGBM Regressor
lgb_regressor = lgb.LGBMRegressor()
lgb_regressor.fit(X_train_features, y_train)
y_train_pred_lgb = lgb_regressor.predict(X_train_features)
y_test_pred_lgb = lgb_regressor.predict(X_test_features)

Learning rate set to 0.03738
0:	learn: 2.1768313	total: 85.8ms	remaining: 1m 25s
1:	learn: 2.1580015	total: 106ms	remaining: 52.7s
2:	learn: 2.1378666	total: 128ms	remaining: 42.6s
3:	learn: 2.1179388	total: 148ms	remaining: 36.8s
4:	learn: 2.0989246	total: 174ms	remaining: 34.7s
5:	learn: 2.0805404	total: 200ms	remaining: 33.1s
6:	learn: 2.0641689	total: 221ms	remaining: 31.3s
7:	learn: 2.0481696	total: 240ms	remaining: 29.8s
8:	learn: 2.0308534	total: 261ms	remaining: 28.7s
9:	learn: 2.0162394	total: 282ms	remaining: 27.9s
10:	learn: 2.0013399	total: 302ms	remaining: 27.1s
11:	learn: 1.9879285	total: 322ms	remaining: 26.5s
12:	learn: 1.9752908	total: 344ms	remaining: 26.1s
13:	learn: 1.9625218	total: 365ms	remaining: 25.7s
14:	learn: 1.9524685	total: 385ms	remaining: 25.3s
15:	learn: 1.9406672	total: 407ms	remaining: 25s
16:	learn: 1.9305682	total: 428ms	remaining: 24.7s
17:	learn: 1.9200557	total: 448ms	remaining: 24.4s
18:	learn: 1.9116652	total: 468ms	remaining: 24.2s
19:	learn: 1

In [20]:
# Calculate evaluation metrics for CatBoost Regressor
mape_train_catboost = mean_absolute_percentage_error(y_train, y_train_pred_catboost)
mape_test_catboost = mean_absolute_percentage_error(y_test, y_test_pred_catboost)
r2_train_catboost = r2_score(y_train, y_train_pred_catboost)
r2_test_catboost = r2_score(y_test, y_test_pred_catboost)
rmse_train_catboost = mean_squared_error(y_train, y_train_pred_catboost, squared=False)
rmse_test_catboost = mean_squared_error(y_test, y_test_pred_catboost, squared=False)
mae_train_catboost = mean_absolute_error(y_train, y_train_pred_catboost)
mae_test_catboost = mean_absolute_error(y_test, y_test_pred_catboost)

# Calculate evaluation metrics for LGBM Regressor
mape_train_lgb = mean_absolute_percentage_error(y_train, y_train_pred_lgb)
mape_test_lgb = mean_absolute_percentage_error(y_test, y_test_pred_lgb)
r2_train_lgb = r2_score(y_train, y_train_pred_lgb)
r2_test_lgb = r2_score(y_test, y_test_pred_lgb)
rmse_train_lgb = mean_squared_error(y_train, y_train_pred_lgb, squared=False)
rmse_test_lgb = mean_squared_error(y_test, y_test_pred_lgb, squared=False)
mae_train_lgb = mean_absolute_error(y_train, y_train_pred_lgb)
mae_test_lgb = mean_absolute_error(y_test, y_test_pred_lgb)

# Calculate evaluation metrics for LGBM Regressor
mape_train_ridge = mean_absolute_percentage_error(y_train, y_train_pred_ridge)
mape_test_ridge = mean_absolute_percentage_error(y_test, y_test_pred_ridge)
r2_train_ridge = r2_score(y_train, y_train_pred_ridge)
r2_test_ridge = r2_score(y_test, y_test_pred_ridge)
rmse_train_ridge = mean_squared_error(y_train, y_train_pred_ridge, squared=False)
rmse_test_ridge = mean_squared_error(y_test, y_test_pred_ridge, squared=False)
mae_train_ridge = mean_absolute_error(y_train, y_train_pred_ridge)
mae_test_ridge = mean_absolute_error(y_test, y_test_pred_ridge)




In [21]:
# Print the evaluation metrics for CatBoost Regressor
print("CatBoost Regressor:")
print("Train RMSE:", rmse_train_catboost)
print("Test RMSE:", rmse_test_catboost)
print("Train MAE:", mae_train_catboost)
print("Test MAE:", mae_test_catboost)
print("Train MAPE:", mape_train_catboost)
print("Test MAPE:", mape_test_catboost)
print("Train R^2:", r2_train_catboost)
print("Test R^2:", r2_test_catboost)

# Print the evaluation metrics for LGBM Regressor
print("LGBM Regressor:")
print("Train RMSE:", rmse_train_lgb)
print("Test RMSE:", rmse_test_lgb)
print("Train MAE:", mae_train_lgb)
print("Test MAE:", mae_test_lgb)
print("Train MAPE:", mape_train_lgb)
print("Test MAPE:", mape_test_lgb)
print("Train R^2:", r2_train_lgb)
print("Test R^2:", r2_test_lgb)

# Print the evaluation metrics for ridge Regressor
print("ridge Regressor:")
print("Train RMSE:", rmse_train_ridge)
print("Test RMSE:", rmse_test_ridge)
print("Train MAE:", mae_train_ridge)
print("Test MAE:", mae_test_ridge)
print("Train MAPE:", mape_train_ridge)
print("Test MAPE:", mape_test_ridge)
print("Train R^2:", r2_train_ridge)
print("Test R^2:", r2_test_ridge)

CatBoost Regressor:
Train RMSE: 0.7047060165044856
Test RMSE: 2.04699326633643
Train MAE: 0.46314914704796506
Test MAE: 1.5417929805658268
Train MAPE: 10187720444745.334
Test MAPE: 234986393979692.2
Train R^2: 0.8974384579128414
Test R^2: 0.21006804197605544
LGBM Regressor:
Train RMSE: 0.731515690203913
Test RMSE: 2.0996097454822875
Train MAE: 0.4085660881717981
Test MAE: 1.5878139474182469
Train MAPE: 11955847834380.027
Test MAPE: 220624974373482.84
Train R^2: 0.8894863622133717
Test R^2: 0.16893686863336843
ridge Regressor:
Train RMSE: 1.8418263217236346
Test RMSE: 2.01962836930792
Train MAE: 1.4013826056950334
Test MAE: 1.550536153400811
Train MAPE: 80986376786695.62
Test MAPE: 272777101953296.12
Train R^2: 0.29940677388062653
Test R^2: 0.2310470254763014
